In [ ]:
import os
import time
import pickle
import pandas as pd
import numpy as np
import glob

# 4. B_test_3_drop.csv에서 Feature 11개 추출

## 01. 전처리 : 스트림 파일 생성

In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np
import glob

In [2]:
# dataset = 피쳐 추출할 파일 
dataset = pd.read_csv('./B_test2_drop.csv',warn_bad_lines=False, error_bad_lines=False, low_memory=False)

In [3]:
dataset.head()

,_ws.col.UTCtime,_ws.col.Protocol,ip.src,ip.dst,tcp.srcport,tcp.dstport,tcp.len,tcp.seq,tcp.ack,udp.srcport,...,http.user_agent,http.connection,http.host,http.response.code,http.server,http.content_type,http.content_length,http.cache_control,SET,_merge
0,07:48:34.499147,TCP,222.104.14.117,10.89.32.47,53258.0,443.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only
1,07:48:34.515154,TLSv1.2,222.104.14.117,10.89.32.47,53258.0,443.0,677.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only
2,07:48:34.534202,TCP,222.104.14.117,10.89.32.47,53258.0,443.0,0.0,678.0,182.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only
3,07:48:34.536717,TLSv1.2,222.104.14.117,10.89.32.47,53258.0,443.0,693.0,678.0,182.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only
4,07:48:34.555843,TCP,222.104.14.117,10.89.32.47,53258.0,443.0,0.0,1371.0,363.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only


In [4]:
# 전처리1. ip.src가 없는 경우를 제외시킴
dataset = dataset[dataset['ip.src'].notnull()]
print(len(dataset))

147230


In [5]:
# 시간 순 정렬

ts = time.time()
print("before data size : " , len(dataset))

dataset = dataset.sort_values(by=['_ws.col.UTCtime'])

print(dataset['_ws.col.Protocol'].value_counts)
print("연산시간 : ",time.time() - ts)
dataset.head()

before data size :  147230
<bound method IndexOpsMixin.value_counts of 0        TCP
447      TCP
19       TCP
44       TCP
100      TCP
        ... 
48642    TCP
48643    TCP
48644    TCP
48645    TCP
48646    TCP
Name: _ws.col.Protocol, Length: 147230, dtype: object>
연산시간 :  0.09978580474853516


,_ws.col.UTCtime,_ws.col.Protocol,ip.src,ip.dst,tcp.srcport,tcp.dstport,tcp.len,tcp.seq,tcp.ack,udp.srcport,...,http.user_agent,http.connection,http.host,http.response.code,http.server,http.content_type,http.content_length,http.cache_control,SET,_merge
0,07:48:34.499147,TCP,222.104.14.117,10.89.32.47,53258.0,443.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only
447,07:48:34.499948,TCP,211.179.209.170,10.89.32.66,42707.0,443.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211.179.209.170 10.89.32.66,left_only
19,07:48:34.500350,TCP,222.104.14.117,10.89.32.47,53254.0,443.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only
44,07:48:34.502320,TCP,222.104.14.117,10.89.32.47,53256.0,443.0,0.0,1.0,182.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only
100,07:48:34.505378,TCP,222.104.14.117,10.89.32.47,53255.0,443.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only


In [76]:
# 전처리3. SET1 = src:port SET2 = dst:port    ====>>>  SET3 src:port dst:port

print(len(dataset))
# tcp.srcport & tcp.dstport astype(int) => 80.0 float으로 되어있는 데이터를 int형으로 변환
dataset['tcp.srcport'] = dataset['tcp.srcport'].astype(int)
dataset['tcp.dstport'] = dataset['tcp.dstport'].astype(int)

# SET1, SET2를 ip:port로 구성
dataset['SET1'] = dataset['ip.src'].str.cat(dataset['tcp.srcport'].astype(str), sep="_")
dataset['SET2'] = dataset['ip.dst'].str.cat(dataset['tcp.dstport'].astype(str), sep="_")


# SET3 는 SET1(공백)SET2로 만듦
dataset['SET3'] = dataset['SET1'].str.cat(dataset['SET2'], sep=" ")
# new_data['SET4'] = new_data['ip.src'].str.cat(new_data['ip.dst'], sep=" ")


print(len(dataset["SET3"]))


147230
147230


In [77]:
dataset.head()

,_ws.col.UTCtime,_ws.col.Protocol,ip.src,ip.dst,tcp.srcport,tcp.dstport,tcp.len,tcp.seq,tcp.ack,udp.srcport,...,http.response.code,http.server,http.content_type,http.content_length,http.cache_control,SET,_merge,SET1,SET2,SET3
0,07:48:34.499147,TCP,222.104.14.117,10.89.32.47,53258,443,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only,222.104.14.117_53258,10.89.32.47_443,222.104.14.117_53258 10.89.32.47_443
447,07:48:34.499948,TCP,211.179.209.170,10.89.32.66,42707,443,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,211.179.209.170 10.89.32.66,left_only,211.179.209.170_42707,10.89.32.66_443,211.179.209.170_42707 10.89.32.66_443
19,07:48:34.500350,TCP,222.104.14.117,10.89.32.47,53254,443,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only,222.104.14.117_53254,10.89.32.47_443,222.104.14.117_53254 10.89.32.47_443
44,07:48:34.502320,TCP,222.104.14.117,10.89.32.47,53256,443,0.0,1.0,182.0,NaN,...,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only,222.104.14.117_53256,10.89.32.47_443,222.104.14.117_53256 10.89.32.47_443
100,07:48:34.505378,TCP,222.104.14.117,10.89.32.47,53255,443,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,222.104.14.117 10.89.32.47,left_only,222.104.14.117_53255,10.89.32.47_443,222.104.14.117_53255 10.89.32.47_443


In [78]:
dataset['SET3']

0         222.104.14.117_53258 10.89.32.47_443
447      211.179.209.170_42707 10.89.32.66_443
19        222.104.14.117_53254 10.89.32.47_443
44        222.104.14.117_53256 10.89.32.47_443
100       222.104.14.117_53255 10.89.32.47_443
                         ...                  
48642    121.152.231.194_40528 10.89.32.47_443
48643    121.152.231.194_40528 10.89.32.47_443
48644    121.152.231.194_40528 10.89.32.47_443
48645    121.152.231.194_40528 10.89.32.47_443
48646    121.152.231.194_40528 10.89.32.47_443
Name: SET3, Length: 147230, dtype: object

In [79]:
# 전처리4. IP와 포트기반으로 데이터를 자름 

# new_data를 SET3 기준으로 groupby를 함 (group_data)
group_data = dataset.groupby(dataset['SET3'])

# group_data 크기 반환  => 해당 개수만큼 csv파일 생성
test = group_data
print(len([ip for ip, ip_list in test]),"개수만큼 split data생성")

6168 개수만큼 split data생성


In [80]:
# <<<<<<<<<<<<<<<<<<폴더 명 생성 및 변경 >>>>>>>>>>>>>>>>>>>>>>>
# 전처리4. GROUP_DATA2로 CSV생성 

###### split csv ######## -> 해당 폴더에 파일이 몇 개 생겼는지 확인해서 진행정도 확인 가능
print("START")
start = time.time()
## -> 스트림 파일 생성할 폴더 이름을 변경해줘야함
for ip, ip_list in test :
    path = './B_attack_split/' + ip_list['SET3'].values[0] + '.csv'
    # path = './test2/' + ip_list['set'].values[0] + '.csv'
    ip_list.to_csv(path, index=False)
    
print(time.time() - start)
print("FINISH")
###### split csv ########


START
10.530386686325073
FINISH


## 02.Feature 추출

In [27]:
def function_00(filename):
    return filename

In [28]:
# 수정 후 feature1 : 스트림 통신시간 (스트림 마지막시간 - 스트림 시작시간) 
def Fl_dur(stream) : # time sort 
    if (type(stream) != int) :
#         df = pd.concat([stream1,stream2])
        time = stream['_ws.col.UTCtime']
        return (max(pd.to_timedelta(time)).total_seconds() - min(pd.to_timedelta(time)).total_seconds())
    elif (type(stream) != int) :
        return -1
    elif len(stream)==1:
        return -1
    

In [29]:
# 수정 후 feature2 : 외부에서 내부방향으로 가는 패킷 개수
def tot_fw_pk(stream) : # tot_bw_pk backwar9d 방향의 패킷의 수
    ln_ex_pkt_num=[]
    try:
        ln_ex_pkt_num = (len(stream))
#     ln_ex_pkt_sum = sum(ln_ex_pkt_num)
    except:
        ln_ex_pkt_num = -1
    return (ln_ex_pkt_num )
        

In [30]:
# 수정 후 feature4 : 외부에서 내부방향으로 가는 두 패킷사이의 시간의 총합

def fw_iat_tot(stream): 
    if type(stream) == int:
        return -1
    else:
        time = pd.to_timedelta(stream["_ws.col.UTCtime"]).dt.total_seconds()
        time_sorted = sorted(list(time))
        return (max(time_sorted)-min(time_sorted))

In [31]:
# 수정 후 feature5 : 외부에서 내부방향으로 가는 두 패킷사이의 시간의 평균

def fw_iat_avg(stream): #Forward(외부>내부) 방향으로 가는 두 패킷 사이의 시간의 평균
    if type(stream) == int:
        return -1
    else:
        time = pd.to_timedelta(stream["_ws.col.UTCtime"]).dt.total_seconds()
        time_sorted = sorted(list(time))
#     print(np.mean(time_sorted))
        return (np.mean(time_sorted))

In [32]:
# 수정 후 feature6 : 외부에서 내부방향으로 가는 두 패킷사이의 시간의 표준편차
def fw_iat_std(stream): #Forward(외부>내부) 방향으로 가는 두 패킷 사이의 시간의 표준편차 
    if type(stream) == int:
        return -1
    else:
        if len(stream) == 1:
            return -1
        time = pd.to_timedelta(stream["_ws.col.UTCtime"]).dt.total_seconds()
        time_sorted = sorted(list(time))
        list_std = []
        for i in range(1, len(time_sorted)):
            list_std.append(time_sorted[i]-time_sorted[i-1]) 
    #     print(list_std)
#     print(np.std(list_std))
    return (np.std(list_std))

In [33]:
# 수정 후 feature7 : 외부에서 내부방향으로 가는 두 패킷사이의 시간의 최대값

def fw_iat_max(stream): # 외부 -> 내부
    if type(stream) == int:
        return -1
    else:
        time = pd.to_timedelta(stream["_ws.col.UTCtime"]).dt.total_seconds()
        if len(stream) == 1:
            return -1
        else:
            time_sorted = sorted(list(time))
            list_std = []
            for i in range(1, len(time_sorted)):
                list_std.append(time_sorted[i]-time_sorted[i-1])
            return max(list_std)

In [34]:
# 수정 후 feature8 : 외부에서 내부방향으로 가는 두 패킷사이의 시간의 최소값

def fw_iat_min(stream):
    if type(stream) == int:
        return -1
    if len(stream) == 1:
        return -1
    else:
        time = pd.to_timedelta(stream["_ws.col.UTCtime"]).dt.total_seconds()
        time_sorted = sorted(list(time))
        list_std = []
        for i in range(1, len(time_sorted)):
            list_std.append(time_sorted[i]-time_sorted[i-1]) 
    return min(list_std)

In [35]:
# 수정 후 feature14 : 외부에서 내부방향으로 가는 1초당 패킷의 개수

def fw_pkt_s(stream) : # max
    try:
        stream["seconds"] = round(pd.to_timedelta(stream["_ws.col.UTCtime"]).dt.total_seconds())
        time_set = sorted(list(set(stream["seconds"])))
        result = []
        for i in range(len(time_set)):
            result.append(len(stream[stream['seconds'] == time_set[i]]))
        #print(result)
        
        return (max(result))
#         return result
    except:
        return -1

In [36]:
# set2용 피처추가 rdp 사용여부 체크

def check_rdp(stream):
    try:
        if stream['tcp.dstport'] == 3389:
            return 1
        else:
            return 0
    except:
        return 0

In [53]:
# # set2용 피처추가 feature 15: http.user_agent에 bot이 있는지

# def is_in_bot(stream):
# #     try:
# #         stream['http.user_agent']
# #     print(stream.isnull().sum())
#     stream['http.user_agent']= stream['http.user_agent'].fillna("na")
# #     stream.reindex()
#     if ((stream[stream['http.user_agent'].str.lower().str.contains("bot").notnull()]).shape[0]!=0):
#         return 1
#     else :
#         return 0
# #         return 2

In [68]:
def is_in_bot(stream):
    stream['http.user_agent'] = stream['http.user_agent'].fillna("na")
    if len(stream[stream['http.user_agent'].str.lower().str.contains("bot")==True]):
        return 1
    else :
            return 0

In [69]:
def is_in_ares(stream):
    stream['http.server'] = stream['http.server'].fillna("na")
    if len(stream[stream['http.server'].str.lower().str.contains("ares")==True]):
        return 1
    else :
            return 0

In [100]:
# Feature 추출

result =[]

files = glob.glob( './A_total_normal_split/*.csv') # 스트림 파일들이 있는 폴더로 경로설정

for file in files:
    
    temp=[]
    filename = file.split("\\")[1]
    
    try:
        stream = pd.read_csv(file)  
    except:
        stream = -1

    temp.append(function_00(filename)) 
    temp.append(Fl_dur(stream))           # Feature 1
    temp.append(tot_fw_pk(stream))        # Feature 2
    temp.append(fw_iat_tot(stream))       # Feature 4
    temp.append(fw_iat_avg(stream))       # Feature 5
    temp.append(fw_iat_std(stream))       # Feature 6
    temp.append(fw_iat_max(stream))       # Feature 7
    temp.append(fw_iat_min(stream))       # Feature 8
    temp.append(fw_pkt_s(stream))         # Feature 14
    temp.append(is_in_bot(stream))
    temp.append(is_in_ares(stream))
    temp.append(check_rdp(stream))
    result.append(temp)

    
print(result)
        

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [102]:
result

[['1.1.70.73_80 192.168.10.19_57627.csv',
  1.6652800000010757,
  5,
  1.6652800000010757,
  52627.4083948,
  0.6214510901985625,
  1.4852260000043316,
  0.0001919999995152466,
  4,
  0,
  0,
  0],
 ['1.1.70.73_80 192.168.10.19_57628.csv',
  0.3926980000032927,
  4,
  0.39269799999601673,
  52627.876973000006,
  0.09256624306156158,
  0.19795099999464583,
  4.0000013541430235e-06,
  4,
  0,
  0,
  0],
 ['1.1.70.73_80 192.168.10.19_57629.csv',
  0.36161600000195904,
  4,
  0.36161600000195904,
  52627.8458045,
  0.08523462849572855,
  0.18178500000067288,
  4.0000013541430235e-06,
  4,
  0,
  0,
  0],
 ['1.1.70.73_80 192.168.10.19_57630.csv',
  0.3904529999999795,
  4,
  0.3904529999999795,
  52627.87519825,
  0.09203201671472627,
  0.196101000001363,
  2.0000006770715117e-06,
  4,
  0,
  0,
  0],
 ['1.1.70.73_80 192.168.10.19_57631.csv',
  0.383269999998447,
  4,
  0.383270000005723,
  52627.86732875,
  0.09033876615877635,
  0.1923460000034538,
  1.0000003385357559e-06,
  4,
  0,
  0,

In [103]:
# temp = pd.read_csv("157.55.39.78_4242 10.")

In [104]:
# result to data

features = pd.DataFrame(result, columns = ['SET','fl_dur','tot_fw_pk','fw_iat_tot','fw_iat_avg','fw_iat_std','fw_iat_max','fw_iat_min','fw_pkt_s','is_in_bot','is_in_ares','check_rdp'])
len(features)

features.shape

(216289, 12)

In [105]:
features['is_in_bot'].value_counts()

0    216289
Name: is_in_bot, dtype: int64

In [106]:
features['is_in_ares'].value_counts()

0    216289
Name: is_in_ares, dtype: int64

In [107]:
features.to_pickle('./B_train.pkl')

In [108]:
result1 = pd.read_pickle('./B_train.pkl')
len(result1)

216289

In [109]:
result1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216289 entries, 0 to 216288
Data columns (total 12 columns):
SET           216289 non-null object
fl_dur        216289 non-null float64
tot_fw_pk     216289 non-null int64
fw_iat_tot    216289 non-null float64
fw_iat_avg    216289 non-null float64
fw_iat_std    216289 non-null float64
fw_iat_max    216289 non-null float64
fw_iat_min    216289 non-null float64
fw_pkt_s      216289 non-null int64
is_in_bot     216289 non-null int64
is_in_ares    216289 non-null int64
check_rdp     216289 non-null int64
dtypes: float64(6), int64(5), object(1)
memory usage: 19.8+ MB


In [110]:
result1.isnull().sum()

SET           0
fl_dur        0
tot_fw_pk     0
fw_iat_tot    0
fw_iat_avg    0
fw_iat_std    0
fw_iat_max    0
fw_iat_min    0
fw_pkt_s      0
is_in_bot     0
is_in_ares    0
check_rdp     0
dtype: int64

In [111]:
# 라벨링 해주기!!!!!!!!!!!!!!!!!- test 파일 라벨링 생략
#정상은 0, 악성은 1  로 수정해야함

# result1['label'] = 1    #악성파일 라벨링
# result1['label'] = 0   #정상파일 라벨링

In [112]:
result1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216289 entries, 0 to 216288
Data columns (total 12 columns):
SET           216289 non-null object
fl_dur        216289 non-null float64
tot_fw_pk     216289 non-null int64
fw_iat_tot    216289 non-null float64
fw_iat_avg    216289 non-null float64
fw_iat_std    216289 non-null float64
fw_iat_max    216289 non-null float64
fw_iat_min    216289 non-null float64
fw_pkt_s      216289 non-null int64
is_in_bot     216289 non-null int64
is_in_ares    216289 non-null int64
check_rdp     216289 non-null int64
dtypes: float64(6), int64(5), object(1)
memory usage: 19.8+ MB


In [113]:
result1.to_csv("./A_total_normal_fea12.csv", index = False)

In [114]:
result1.isnull().sum()

SET           0
fl_dur        0
tot_fw_pk     0
fw_iat_tot    0
fw_iat_avg    0
fw_iat_std    0
fw_iat_max    0
fw_iat_min    0
fw_pkt_s      0
is_in_bot     0
is_in_ares    0
check_rdp     0
dtype: int64